In [4]:
import scrapy
import re
from scrapy.crawler import CrawlerProcess

In [ ]:
# let's go hunting for mobile phones - picked semirandom. Following pages were set as root and processed:
# 1) https://rozetka.com.ua/mobile-phones/c80003/ - mobile phones
# 2) https://rozetka.com.ua/notebooks/c80004/filter/ - laptops
# 3) https://bt.rozetka.com.ua/refrigerators/c80125/filter/ - fridges
# 4) https://rozetka.com.ua/all-tv/c80037/ - tvs
# 5) https://hard.rozetka.com.ua/mouses/c80172/ - computer mice
# 6) https://rozetka.com.ua/headphones/c80027/ - headphones
# 7) https://rozetka.com.ua/usb-flash-memory/c80045/ - usb sticks
# 8) https://rozetka.com.ua/notebook-bags/c80036/ - laptop bags
# 9) https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/ - laptop keyboards
# Structure of page <-> selectors mapping
# #block_with_goods - main content here
# all pages links wrapper '.paginator-catalog' 
# next page link '.paginator-catalog-l-i.active  + .paginator-catalog-l-i a'
# link to review page .g-rating-reviews-link (filter :has(.g-rating-reviews)) + add to path sort=from_buyer/
# On comments page: 
    # HEH PRICES ARE NOT SHOWN IF YOU HAVE JS DISABLED
    # itemprop="review" - single review holder
        # itemprop="name" #attr(content) - product name
        # itemprop="datePublished" #attr(content)
        # itemprop="author" #innerText - author
        # itemprop="reviewRating" itemprop="worstRating" #attr(content)
        # itemprop="reviewRating" itemprop="bestRating" #attr(content)
        # itemprop="reviewRating" itemprop="ratingValue" #attr(content)
        # .pp-comments-author-good-vote#innerText - extra info % of considering this useful
        # .pp-review-text#innerHTML - review markup
        # .pp-review-vote-negative .pp-review-vote-count#text - negative count of votes for review
        # .pp-review-vote-positive .pp-review-vote-count#text - positive count of votes for review
    # pagination:
        # .paginator-catalog-l-i.active + .paginator-catalog-l-i a
    # can stop if no '[itemprop="review"] [itemprop="ratingValue"]' on the page

In [2]:
startUrl = "https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/"
innerPagesPushed = 0
innerReviewsPushed = 0
innerReviewsPagesPushed = 0
class RozetkaParseScrapper(scrapy.Spider):
    name = 'rozetka.com.ua'
    start_urls = [startUrl]

    def parse(self, response):
        global innerReviewsPushed, innerPagesPushed
        review_links = response.xpath(".//*[(contains(concat(' ', normalize-space(./@class), ' '), ' g-rating-reviews-link ') and .//*[contains(concat(' ', normalize-space(./@class), ' '), ' g-rating-reviews ')])]/@href").extract()
        for link in review_links:
            new_link = link + "sort=from_buyer/"
            yield response.follow(link, self.parseReviews)

        for next_page in response.xpath(".//*[(contains(concat(' ', normalize-space(./@class), ' '), ' paginator-catalog-l-i ') and contains(concat(' ', normalize-space(./@class), ' '), ' active '))]/following-sibling::*[1]/self::*[contains(concat(' ', normalize-space(./@class), ' '), ' paginator-catalog-l-i ')]//a/@href").extract():
            yield response.follow(next_page, self.parse)

    def parseReviews(self, response):
        global innerReviewsPagesPushed
        if (len(response.css('[itemprop="review"] [itemprop="ratingValue"]').extract()) == 0):
            return
        reviews = response.css('[itemprop="review"]')
        reviews.extract()

        for review in reviews:
            rating_value = review.css('[itemprop="ratingValue"]::attr(content)').extract()
            if (len(rating_value) == 0):
                return
            vote_extra = review.css('.pp-comments-author-good-vote::text').extract()
            vote_neg = review.css('.pp-review-vote-negative .pp-review-vote-count::text').extract()
            vote_pos = review.css('.pp-review-vote-positive .pp-review-vote-count::text').extract()
            yield {
                'product_name': review.css('[itemprop="name"]::attr(content)').extract()[0],
                'date_published': review.css('[itemprop="datePublished"]::attr(content)').extract()[0],
                'author': review.css('[itemprop="author"]::text').extract()[0],
                'rating_min': review.css('[itemprop="worstRating"]::attr(content)').extract()[0],
                'rating_max': review.css('[itemprop="bestRating"]::attr(content)').extract()[0],
                'rating_value': rating_value[0],
                'vote_extra_info': (vote_extra and vote_extra[0]) or '',
                'vote_negative': (vote_neg and vote_neg[0]) or '',
                'vote_positive': (vote_pos and vote_pos[0]) or '',
                'review_markup': re.sub("[\n\r\t]{1,}", "\n", "".join(review.css('.pp-review-text *::text').extract()).strip("\n\t\r"))
            }
            
        next_page = response.xpath(".//*[(contains(concat(' ', normalize-space(./@class), ' '), ' paginator-catalog-l-i ') and contains(concat(' ', normalize-space(./@class), ' '), ' active '))]/following-sibling::*[1]/self::*[contains(concat(' ', normalize-space(./@class), ' '), ' paginator-catalog-l-i ')]//a/@href").extract()
        if next_page:
            yield response.follow(next_page[0], self.parseReviews)


In [3]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'DOWNLOAD_DELAY': '0.25', # '0.25',
    'FEED_FORMAT': 'jl',
    'FEED_URI': 'keyboards.jsonline',
    'FEED_EXPORT_ENCODING': 'utf-8'
})

process.crawl(RozetkaParseScrapper)
process.start() # the script will block here until the crawling is finished

2018-04-04 22:46:35 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: scrapybot)
2018-04-04 22:46:35 [scrapy.utils.log] INFO: Versions: lxml 4.1.0.0, libxml2 2.9.6, cssselect 1.0.1, parsel 1.4.0, w3lib 1.19.0, Twisted 17.9.0, Python 3.6.4 (default, Mar 18 2018, 16:25:25) - [GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.39.2)], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.2, Platform Darwin-17.4.0-x86_64-i386-64bit
2018-04-04 22:46:35 [scrapy.crawler] INFO: Overridden settings: {'DOWNLOAD_DELAY': '0.25', 'FEED_EXPORT_ENCODING': 'utf-8', 'FEED_FORMAT': 'jl', 'FEED_URI': 'keyboards.jsonline', 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2018-04-04 22:46:35 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2018-04-04 22:46:35 [scrapy.mid

2018-04-04 22:46:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/32008103/p32008103/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=2/)
2018-04-04 22:46:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/32007431/p32007431/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=2/)
2018-04-04 22:46:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/32007231/p32007231/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=2/)
2018-04-04 22:46:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/32006559/p32006559/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=2/)
2018-04-04 22:46:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/28247705/p28247705/comments/#t_comments> (referer: https://rozetka.com.ua/klav

2018-04-04 22:46:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/20913957/p20913957/comments/> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=4/)
2018-04-04 22:46:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/powerplant_kb310586/p27718921/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=4/)
2018-04-04 22:46:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/powerplant_kb310944/p27719009/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=4/)
2018-04-04 22:46:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/powerplant_kb310210/p27718825/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=4/)
2018-04-04 22:46:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/20914048/p20914048/comments/> (referer: https://rozetka.

2018-04-04 22:47:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/28251545/p28251545/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=6/)
2018-04-04 22:47:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/28251569/p28251569/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=6/)
2018-04-04 22:47:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/28250257/p28250257/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=6/)
2018-04-04 22:47:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/28251233/p28251233/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=6/)
2018-04-04 22:47:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/28249849/p28249849/comments/#t_comments> (referer: https://rozetka.com.ua/klav

2018-04-04 22:47:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/32004511/p32004511/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=8/)
2018-04-04 22:47:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/32004527/p32004527/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=8/)
2018-04-04 22:47:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=9/> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=8/)
2018-04-04 22:47:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/32004463/p32004463/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=8/)
2018-04-04 22:47:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/32004479/p32004479/comments/#t_comments> (referer: https://rozetka.com.ua/k

...

2018-04-04 23:03:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/32007055/p32007055/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=10/)
2018-04-04 23:03:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/32006999/p32006999/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=10/)
2018-04-04 23:03:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/32006983/p32006983/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=10/)
2018-04-04 23:03:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/32006911/p32006911/comments/> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=10/)
2018-04-04 23:03:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/32006991/p32006991/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-

2018-04-04 23:04:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://rozetka.com.ua/29704599/p29704599/comments/>
{'product_name': 'Клавиатура LENOVO IdeaPad 100-15 ( RU Black ). Русская раскладка. Цвет Черный. (64218)', 'date_published': '2018-3-07', 'author': '\nСергей', 'rating_min': '1', 'rating_max': '5', 'rating_value': '5', 'vote_extra_info': '', 'vote_negative': '', 'vote_positive': '', 'review_markup': 'Легла в ноут, как родная.'}
2018-04-04 23:04:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/29698751/p29698751/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=7/)
2018-04-04 23:04:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/29698071/p29698071/comments/> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=7/)
2018-04-04 23:04:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/29697575/p29697575/comments/#t_comments> (referer: http

2018-04-04 23:04:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/powerplant_kb311545/p27719089/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=4/)
2018-04-04 23:04:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/powerplant_kb311675/p27719137/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=4/)
2018-04-04 23:04:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/powerplant_kb311668/p27719129/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=4/)
2018-04-04 23:04:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/powerplant_kb310807/p27718993/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=4/)
2018-04-04 23:04:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/powerplant_kb310692/p27718977/comm

2018-04-04 23:04:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/32006607/p32006607/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=2/)
2018-04-04 23:04:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/32006615/p32006615/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=2/)
2018-04-04 23:04:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/28252873/p28252873/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=2/)
2018-04-04 23:04:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/28238113/p28238113/comments/#t_comments> (referer: https://rozetka.com.ua/klaviaturi-dlya-noutbukov/c4631383/page=2/)
2018-04-04 23:04:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://rozetka.com.ua/28249305/p28249305/comments/#t_comments> (referer: https://rozetka.com.ua/klav

2018-04-04 23:04:36 [scrapy.extensions.feedexport] INFO: Stored jl feed (30 items) in: keyboards.jsonline
2018-04-04 23:04:36 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 1310659,
 'downloader/request_count': 3518,
 'downloader/request_method_count/GET': 3518,
 'downloader/response_bytes': 180519215,
 'downloader/response_count': 3518,
 'downloader/response_status_count/200': 3518,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2018, 4, 4, 20, 4, 36, 724913),
 'item_scraped_count': 30,
 'log_count/DEBUG': 3549,
 'log_count/INFO': 26,
 'memusage/max': 131387392,
 'memusage/startup': 72048640,
 'request_depth_max': 107,
 'response_received_count': 3518,
 'scheduler/dequeued': 3518,
 'scheduler/dequeued/memory': 3518,
 'scheduler/enqueued': 3518,
 'scheduler/enqueued/memory': 3518,
 'start_time': datetime.datetime(2018, 4, 4, 19, 46, 35, 776409)}
2018-04-04 23:04:36 [scrapy.core.engine] INFO: Spider closed (finished)
